In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
train_text = pd.read_csv('/content/drive/MyDrive/csci_567_project/concatenated_data/train_textmodel.csv', header=None)
train_text = train_text.iloc[:, 1:-1]

In [5]:
train_audio = pd.read_csv('/content/drive/MyDrive/csci_567_project/concatenated_data/train_voice.csv', header=None)
train_audio = train_audio.iloc[:, 1:-1]

In [6]:
def sentiment_to_tensor(s: str):
    if s == 'positive':
        return 0
    elif s == 'neutral':
        return 1
    else:
        return 2

train_labels = pd.read_csv('/content/drive/MyDrive/csci_567_project/train_text.csv')
train_labels = train_labels['Sentiment'].map(lambda x: sentiment_to_tensor(x))

In [7]:
batch_size = 32

class ConcatenatedDataset:
    def __init__(self, text, audio, labels) -> None:
        self.text = text
        self.audio = audio
        self.labels = labels

    def __len__(self) -> int:
        return self.text.shape[0]

    def __getitem__(self, idx):
        np_concatenated = np.concatenate((self.text.iloc[idx].values, self.audio.iloc[idx].values))
        return torch.from_numpy(np_concatenated).type(torch.float32), self.labels.iloc[idx]

train_dataset = ConcatenatedDataset(train_text, train_audio, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [15]:
lr = 0.01

class ConcatenatedNetwork(nn.Module):
    def __init__(self):
        super(ConcatenatedNetwork, self).__init__()

        self.l1 = nn.Linear(train_dataloader.dataset[0][0].shape[0], 128)
        self.l2 = nn.Linear(128, 64)
        self.l3 = nn.Linear(64, 32)
        self.classification = nn.Linear(32, 3)

    def forward(self, x):
        x = self.l1(x)
        x = nn.functional.relu(x)
        x = self.l2(x)
        x = nn.functional.relu(x)
        x = self.l3(x)
        x = nn.functional.relu(x)
        x = self.classification(x)
        x = nn.functional.softmax(x)
        return x

network = ConcatenatedNetwork().to(device=device)

optimizer = torch.optim.AdamW(params=network.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [16]:
# Tunable hyperparameters
num_epochs = 25

valid_loss_min = np.Inf

for epoch in range(num_epochs):
    train_loss = 0.0

    network.train()
    for data, target in train_dataloader:
        optimizer.zero_grad()
        data = data.to(device)
        target = target.to(device)
        output = network(data)

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch + 1} completed')

<ipython-input-15-4053e914cb64>:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.functional.softmax(x)


Epoch 1 completed
Epoch 2 completed
Epoch 3 completed
Epoch 4 completed
Epoch 5 completed
Epoch 6 completed
Epoch 7 completed
Epoch 8 completed
Epoch 9 completed
Epoch 10 completed
Epoch 11 completed
Epoch 12 completed
Epoch 13 completed
Epoch 14 completed
Epoch 15 completed
Epoch 16 completed
Epoch 17 completed
Epoch 18 completed
Epoch 19 completed
Epoch 20 completed
Epoch 21 completed
Epoch 22 completed
Epoch 23 completed
Epoch 24 completed
Epoch 25 completed


In [17]:
dev_text = pd.read_csv('/content/drive/MyDrive/csci_567_project/concatenated_data/dev_textmodel.csv', header=None)
dev_text = dev_text.iloc[:, 1:-1]

In [18]:
dev_audio = pd.read_csv('/content/drive/MyDrive/csci_567_project/concatenated_data/dev_voice.csv', header=None)
dev_audio = dev_audio.iloc[:, 1:-1]

In [19]:
dev_labels = pd.read_csv('/content/drive/MyDrive/csci_567_project/dev_text.csv')
dev_labels = dev_labels['Sentiment'].map(lambda x: sentiment_to_tensor(x))

In [20]:
dev_dataset = ConcatenatedDataset(dev_text, dev_audio, dev_labels)
dev_dataloader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=True)

In [21]:
correct = 0
total = 0

network.eval()
for data, label in dev_dataloader:
    data = data.to(device)
    label = label.to(device)
    output = network(data)
    for o,l in zip(torch.argmax(output,axis = 1),label):
        if o == l:
            correct += 1
        total += 1

<ipython-input-15-4053e914cb64>:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.functional.softmax(x)


In [22]:
print(correct / total)

0.45076784101174344


In [23]:
test_text = pd.read_csv('/content/drive/MyDrive/csci_567_project/concatenated_data/test_textmodel.csv', header=None)
test_text = test_text.iloc[:, 1:-1]

In [24]:
test_audio = pd.read_csv('/content/drive/MyDrive/csci_567_project/concatenated_data/test_voice.csv', header=None)
test_audio = test_audio.iloc[:, 1:-1]

In [27]:
test_labels = pd.read_csv('/content/drive/MyDrive/csci_567_project/test_text.csv')
test_labels = test_labels['Sentiment'].map(lambda x: sentiment_to_tensor(x))

In [28]:
test_dataset = ConcatenatedDataset(test_text, test_audio, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [29]:
correct = 0
total = 0

network.eval()
for data, label in test_dataloader:
    data = data.to(device)
    label = label.to(device)
    output = network(data)
    for o,l in zip(torch.argmax(output,axis = 1),label):
        if o == l:
            correct += 1
        total += 1

<ipython-input-15-4053e914cb64>:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.functional.softmax(x)


In [30]:
print(correct / total)

0.46091954022988507
